# Prerequisites

# AWS Video Understanding on AWS 워크숍에 오신 것을 환영합니다!

워크숍은 기초와 사용 사례, 두 가지 주요 부분으로 구성되어 있습니다. 기초 노트북에서는 노트북 환경을 설정하고, 샘플 비디오를 클립으로 나누어 준비하며, Foundation 모델을 사용하여 비디오 클립에 대한 인사이트를 생성하는 실험을 하게 됩니다. 워크숍의 두 번째 부분에서는 이러한 기초를 바탕으로 다양한 비디오 이해 사용 사례를 해결하게 됩니다. 사용 사례들은 독립적이며 어떤 순서로든 실행할 수 있습니다.

**Part 1: Foundations (필수, 순서대로 실행):**

1. Prerequisites (현재 노트북) (10분)
2. [Visual video segments: frames, shots and scenes](01A-visual-segments-frames-shots-scenes.ipynb) (20분)
3. [Audio segments](01B-audio-segments.ipynb) (10분)

**Part 2: Use cases (선택, 순서 무관):**

기초 노트북을 실행한 후에는 어떤 사용 사례든 선택할 수 있습니다. AWS Workshop 이벤트에서 실행하는 경우, 2시간 세션 동안 기초와 하나의 사용 사례를 완료할 수 있습니다:

* [Ad break detection and contextual Ad targeting](02-ad-breaks-and-contextual-ad-targeting.ipynb) (20분) - 광고 삽입 기회를 식별합니다. 표준 분류법을 사용하여 비디오 콘텐츠를 광고 콘텐츠와 매칭합니다.
* [Video summarization](03-video-summarization.ipynb) (20분) - 긴 비디오에서 짧은 형식의 비디오를 생성합니다.
* [Semantic video search](04-semantic-video-search.ipynb) (20분) - 이미지와 자연어를 사용하여 관련 클립을 찾는 비디오 검색

**Additional Notebooks**

`additional-notebooks` 폴더에 추가 노트북이 있습니다. 이는 주요 튜토리얼의 작업에 대한 대체 구현 튜토리얼입니다.

* [Shot detection with Titan Multimodal Embeddings](./additional-notebooks/01a-shot-detection-with-embeddings.ipynb)

**Resources**

이 워크숍의 활동들은 AWS Solution Guidance를 기반으로 합니다. [Additional Resources](./09-resources.ipynb) 랩에는 관련 참조 아키텍처, 코드 샘플 및 블로그 게시물 링크가 포함되어 있습니다.

# ffmpeg 및 python 패키지 설치

- 비디오 및 이미지 처리를 위한 ffmpeg
- 벡터 저장소를 위한 faiss
- 자막 파일 파싱을 위한 webvtt-py 
- 출력 포맷팅을 위한 termcolor

In [1]:
## install ffmpeg
!sudo apt update -y && sudo apt-get -y install ffmpeg

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease    
Fetched 257 kB in 1s (302 kB/s)                      
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
25 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [2]:
## Make sure ffmpeg is installed
!which ffmpeg

/usr/bin/ffmpeg


In [3]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## SageMaker 기본 리소스 가져오기

In [4]:
import sagemaker

sagemaker_resources = {}
#sagemaker_resources["session"] = sagemaker.Session()
sagemaker_resources["bucket"] = sagemaker.Session().default_bucket()
sagemaker_resources["role"] = sagemaker.get_execution_role()
sagemaker_resources["region"] = sagemaker.Session()._region_name

print(sagemaker_resources)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
{'bucket': 'sagemaker-us-west-2-227211500130', 'role': 'arn:aws:iam::227211500130:role/workshop-SageMakerExecutionRole-WKKoQonyGPqq', 'region': 'us-west-2'}


# AWS 세션 리소스 설정

AWS 호스팅 이벤트에서 실행하는 경우, AWS 리소스가 사전에 생성되어 있으며 아래 셀을 사용하여 CloudFormation 출력에서 이를 로드할 수 있습니다. 자체 계정에서 실행하는 경우, 수동으로 생성한 리소스에서 다음 변수들을 설정해야 합니다.

## Get CloudFormation stack outputs (AWS hosted event only)

In [5]:
import boto3
from IPython.display import JSON
cf = boto3.client(service_name="cloudformation")
stack = response = cf.describe_stacks(StackName='workshop')

In [6]:
JSON(stack)

<IPython.core.display.JSON object>

In [7]:
session = {}
session['bucket'] = next(item["OutputValue"] for item in stack['Stacks'][0]['Outputs'] if item["OutputKey"] == "S3BucketName")
session['MediaConvertRole'] = next(item["OutputValue"] for item in stack['Stacks'][0]['Outputs'] if item["OutputKey"] == "MediaConvertRole")
session["AOSSCollectionEndpoint"] = next(item["OutputValue"] for item in stack['Stacks'][0]['Outputs'] if item["OutputKey"] == "AOSSCollectionEndpoint")

# Amazon Q Developer 찾기

SageMaker Studio의 Jupyter 노트북에는 Amazon Q Developer가 활성화되어 있습니다.

1. Q Developer를 사용하려면 왼쪽 사이드바 메뉴에서 Q Developer 아이콘을 클릭하세요.
<br></br>
<img src="static/images/00-qdev-icon.png" alt="Q Developer Icon" style="width: 600px;"/>
<br></br>
3. 이제 활성화된 사이드 패널이 Amazon Q Developer가 되어야 합니다.
<br></br>
<img src="static/images/00-qdev-sidebar.png" alt="Q Developer Sidebar" style="width: 600px;"/>
<br></br>
5. 질문을 해보면서 사용해보세요. 예를 들어, 다음과 같이 물어볼 수 있습니다: `What kinds of questions can Q developer answer? Be brief.` 다음과 같은 응답을 받을 수 있습니다:
<br></br>
<img src="static/images/00-qdev-skills.png" alt="Q Developer Skills" style="width: 600px;"/>
<br></br>

이 워크숍 전반에 걸쳐, 오류가 발생하거나 코드에 대한 질문이 있을 때 Q를 사용할 수 있습니다.

# 다른 노트북에서 사용할 변수 저장하기

이 데이터는 다음 실습에서 사용할 예정입니다. 이 데이터를 사용하기 위해 이후 노트북에서 사용할 수 있도록 이 변수들을 저장할 것입니다.

In [8]:
%store sagemaker_resources
%store session

Stored 'sagemaker_resources' (dict)
Stored 'session' (dict)


# 워크숍의 다음 섹션으로 계속하기

계속하려면 다음 노트북 [Video segments: frames, shots, and scenes](01A-visual-segments-frames-shots-scenes.ipynb)으로 이동하세요.